# Traffic Light Detection with YOLO on Bosch data set

This script tries to detect traffic lights in the _Bosch Small Traffic Lights Dataset_ with the [YOLO v2 CNN](https://pjreddie.com/darknet/yolo). To make use of tensorflow on Udacity's carla, the github tensorflow implementation made by thtrieu will be used.

To run this script make sure to meet the following preconditions:
1. Download the train data from https://hci.iwr.uni-heidelberg.de/node/6132/download/c2267f98dcef13de059fa8cdb0f8efed.
    - dataset_train_rgb.zip.001	to dataset_train_rgb.zip.004
    - unzip them all together, you will get a _rgb_ directory with the training data
    - the _rgb_ folder and _train.yaml_ has to be in the _TrafficLightDetector/data_ directory
2. Clone the _darkflow_ repo on the same level as _TrafficLightDetection_
    - https://github.com/thtrieu/darkflow.git
    - install as described (you might need to restart the notebook to have access)
    ```
    pip install -e .
    ```
3. In the _darkflow_ repo, run __/test/test_darkflow.py__ to download the pretraind weights-files
    - check if _/bin/tiny-yolo-voc.weights_ exists
    
Your directories should look like this (or if not, you might have to fix some relative paths in the code below):

- TrafficLightDetector
    - cfg
        - tiny-yolo-voc.cfg
        - yolo_bosch.cfg
    - ckpt
    - data
        - rgb
        - train.yaml
    
    - annotations (<- created below)   
    - ...
- darkflow
    - bin
    - darkflow
    - test
    - ...
    
---    
#### Dependencies:
- tensorflow 1.0
- cv2
- lxml
- tqdm
- yaml

In [1]:
import os
import yaml
from lxml import etree as ET
from tqdm import tqdm
from random import shuffle

First, we need to load the data sets annotations from the _train.yaml_ file in the data folder.

In [2]:
data = []
with open("./data/train.yaml", 'r') as stream:
    try:
        data = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
shuffle(data)

Darkflow uses the Pascal VOC annotation format. The next step converts our annotations to this very format.
### Caution: One annotation has an error (ymin > ymax). The training will fail if you don't eliminate it.
#### Check the output below and switch the min and max  values in _train.yaml_ for the faulty annotation. Then run this cell again.

In [3]:
annotation_path = './annotations'

if not os.path.isdir(annotation_path):
    os.makedirs(annotation_path)

TRAINING_DATA = 1500
#TRAINING_DATA = len(data)
cnt = 0

label_count = dict(zip([
    'Red',
    'RedStraight',
    'RedStraightLeft',
    'RedLeft',
    'RedRight',
    'Yellow',
    'Green',
    'GreenStraight',
    'GreenStraightLeft',
    'GreenStraightRight',
    'GreenLeft',
    'GreenRight',
    'off'],
    [0]*13
))

for img in tqdm(data[:TRAINING_DATA], ncols=80):
    root = ET.Element('annotation')
    
    fn = ET.SubElement(root, 'filename')
    fn.text = img['path']
    
    sz = ET.SubElement(root, 'size')
    
    width = ET.SubElement(sz, 'width')
    width.text = '1280'
    
    height = ET.SubElement(sz, 'height')
    height.text = '720'
    
    depth = ET.SubElement(sz, 'depth')
    depth.text = '3'

    for box in img['boxes']:       
        obj = ET.SubElement(root, 'object')
        
        name = ET.SubElement(obj, 'name')
        name.text = box['label']
        
        bb = ET.SubElement(obj, 'bndbox')
        
        xmin = ET.SubElement(bb, 'xmin')
        x_min = box['x_min']
        xmin.text = str(box['x_min'])
        
        xmax = ET.SubElement(bb, 'xmax')
        x_max = box['x_max']
        xmax.text = str(box['x_max'])
        if (x_max - x_min) < 0:
            print("Annotation Error X", img['path'])
        
        ymin = ET.SubElement(bb, 'ymin')
        y_min = box['y_min']
        ymin.text = str(box['y_min'])
        
        ymax = ET.SubElement(bb, 'ymax')
        y_max = box['y_max']
        ymax.text = str(box['y_max'])
        
        if (y_max - y_min) < 0:
            print("Annotation Error Y", img['path'])
        
        label_count[box['label']] += 1

    et = ET.ElementTree(root)
    et.write(os.path.join(annotation_path, str(cnt) + '.xml'), pretty_print=True)
    
    cnt += 1

for label, count in label_count.items():
    print('{:>20}'.format(label), count)

100%|█████████████████████████████████████| 1500/1500 [00:00<00:00, 2315.97it/s]


             RedLeft 335
          GreenRight 1
   GreenStraightLeft 0
               Green 1661
            RedRight 1
                 Red 846
     RedStraightLeft 0
       GreenStraight 8
                 off 229
         RedStraight 2
           GreenLeft 45
  GreenStraightRight 1
              Yellow 135


### Training

Before training make sure
- that the _labels.txt_ file is in the same directory as this notebook
- annotations exist in Pascal VOC format (many xml files named like this -> 0.xml, 1.xml, 2.xml, ...)
- you have executed darkflow/test/test_darkflow.py successfully and the file darkflow/bin/tiny-yolo-voc.weights
- you have a gpu, if not delete the _gpu_ statement in _options_ to run on cpu

In [4]:
import darkflow
from darkflow.net.build import TFNet
import cv2

options = {"model": "./cfg/yolo_bosch.cfg",
           "load": "../darkflow/bin/tiny-yolo-voc.weights",
           "train": True, 
           "annotation": "./annotations/",
           "dataset": "./data/",
           "batch": 4, "trainer": "adam",
           "epoch": 4,
           "lrate": 1e-5,
           "gpu": 1.0}

tfnet = TFNet(options)

tfnet.train()

Parsing ./cfg/tiny-yolo-voc.cfg
Parsing ./cfg/yolo_bosch.cfg
Loading ../darkflow/bin/tiny-yolo-voc.weights ...
Successfully identified 63471556 bytes
Finished in 0.02215266227722168s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 16)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 26, 26, 128)
 Load  |  Yep!  | conv 3x3p1_1  

step 200 - loss 40.16737365722656 - moving ave loss 42.00469380596118
step 201 - loss 39.521915435791016 - moving ave loss 41.75641596894417
step 202 - loss 36.6663932800293 - moving ave loss 41.24741370005268
step 203 - loss 42.92328643798828 - moving ave loss 41.41500097384624
step 204 - loss 37.674503326416016 - moving ave loss 41.04095120910322
step 205 - loss 36.19020080566406 - moving ave loss 40.555876168759305
step 206 - loss 40.332923889160156 - moving ave loss 40.53358094079939
step 207 - loss 39.1485710144043 - moving ave loss 40.39507994815988
step 208 - loss 41.42572784423828 - moving ave loss 40.498144737767724
step 209 - loss 39.03424835205078 - moving ave loss 40.35175509919603
step 210 - loss 37.90156173706055 - moving ave loss 40.10673576298248
step 211 - loss 36.86126708984375 - moving ave loss 39.78218889566861
step 212 - loss 39.09956741333008 - moving ave loss 39.713926747434755
step 213 - loss 37.761940002441406 - moving ave loss 39.51872807293542
step 214 - loss

step 429 - loss 21.22083282470703 - moving ave loss 19.12832794122252
step 430 - loss 19.58156967163086 - moving ave loss 19.17365211426335
step 431 - loss 18.84554672241211 - moving ave loss 19.140841575078227
step 432 - loss 18.283008575439453 - moving ave loss 19.055058275114348
step 433 - loss 16.840524673461914 - moving ave loss 18.833604914949106
step 434 - loss 18.656322479248047 - moving ave loss 18.815876671379
step 435 - loss 21.880184173583984 - moving ave loss 19.1223074215995
step 436 - loss 17.750383377075195 - moving ave loss 18.985115017147074
step 437 - loss 19.48525619506836 - moving ave loss 19.0351291349392
step 438 - loss 18.288345336914062 - moving ave loss 18.960450755136687
step 439 - loss 15.606462478637695 - moving ave loss 18.625051927486787
step 440 - loss 19.893413543701172 - moving ave loss 18.751888089108228
step 441 - loss 19.50928497314453 - moving ave loss 18.82762777751186
step 442 - loss 16.406429290771484 - moving ave loss 18.585507928837824
step 44

step 657 - loss 14.340164184570312 - moving ave loss 12.650799087669284
step 658 - loss 10.225784301757812 - moving ave loss 12.408297609078137
step 659 - loss 10.930639266967773 - moving ave loss 12.2605317748671
step 660 - loss 14.068584442138672 - moving ave loss 12.441337041594258
step 661 - loss 11.44038200378418 - moving ave loss 12.34124153781325
step 662 - loss 10.749258995056152 - moving ave loss 12.18204328353754
step 663 - loss 11.538010597229004 - moving ave loss 12.117640014906687
step 664 - loss 8.522533416748047 - moving ave loss 11.758129355090825
step 665 - loss 10.372282028198242 - moving ave loss 11.619544622401566
step 666 - loss 14.99256706237793 - moving ave loss 11.956846866399202
step 667 - loss 11.58044719696045 - moving ave loss 11.919206899455327
step 668 - loss 10.394186019897461 - moving ave loss 11.76670481149954
step 669 - loss 10.856106758117676 - moving ave loss 11.675645006161355
step 670 - loss 12.956426620483398 - moving ave loss 11.80372316759356
st

step 886 - loss 7.222723960876465 - moving ave loss 9.659216387394611
step 887 - loss 9.045461654663086 - moving ave loss 9.59784091412146
step 888 - loss 9.703289985656738 - moving ave loss 9.608385821274986
step 889 - loss 8.711042404174805 - moving ave loss 9.518651479564967
step 890 - loss 11.0360689163208 - moving ave loss 9.67039322324055
step 891 - loss 9.329157829284668 - moving ave loss 9.636269683844962
step 892 - loss 10.335226058959961 - moving ave loss 9.706165321356462
step 893 - loss 13.63393783569336 - moving ave loss 10.098942572790152
step 894 - loss 6.113086700439453 - moving ave loss 9.700356985555082
step 895 - loss 9.170238494873047 - moving ave loss 9.647345136486878
step 896 - loss 8.417801856994629 - moving ave loss 9.524390808537653
step 897 - loss 14.078938484191895 - moving ave loss 9.979845576103077
step 898 - loss 11.99506950378418 - moving ave loss 10.181367968871188
step 899 - loss 8.271688461303711 - moving ave loss 9.99040001811444
step 900 - loss 10.8

step 1116 - loss 5.1029276847839355 - moving ave loss 8.943643121411931
step 1117 - loss 5.6400885581970215 - moving ave loss 8.613287665090441
step 1118 - loss 7.9484968185424805 - moving ave loss 8.546808580435645
step 1119 - loss 12.390486717224121 - moving ave loss 8.931176394114493
step 1120 - loss 7.519107818603516 - moving ave loss 8.789969536563396
step 1121 - loss 4.528887748718262 - moving ave loss 8.363861357778882
step 1122 - loss 6.650932312011719 - moving ave loss 8.192568453202167
step 1123 - loss 11.443817138671875 - moving ave loss 8.517693321749137
step 1124 - loss 10.028063774108887 - moving ave loss 8.668730366985113
step 1125 - loss 7.444459915161133 - moving ave loss 8.546303321802714
Finish 3 epoch(es)
step 1126 - loss 8.652013778686523 - moving ave loss 8.556874367491096
step 1127 - loss 9.164424896240234 - moving ave loss 8.61762942036601
step 1128 - loss 9.744048118591309 - moving ave loss 8.73027129018854
step 1129 - loss 15.780874252319336 - moving ave loss 

step 1344 - loss 3.8837666511535645 - moving ave loss 9.345523935786114
step 1345 - loss 6.313348770141602 - moving ave loss 9.042306419221662
step 1346 - loss 11.098194122314453 - moving ave loss 9.247895189530942
step 1347 - loss 6.832771301269531 - moving ave loss 9.0063828007048
step 1348 - loss 7.17290735244751 - moving ave loss 8.823035255879072
step 1349 - loss 11.847370147705078 - moving ave loss 9.125468745061672
step 1350 - loss 21.185588836669922 - moving ave loss 10.331480754222497
step 1351 - loss 4.663103103637695 - moving ave loss 9.764642989164019
step 1352 - loss 7.950299263000488 - moving ave loss 9.583208616547667
step 1353 - loss 10.308107376098633 - moving ave loss 9.655698492502765
step 1354 - loss 7.764311790466309 - moving ave loss 9.46655982229912
step 1355 - loss 8.641754150390625 - moving ave loss 9.38407925510827
step 1356 - loss 9.16230297088623 - moving ave loss 9.361901626686066
step 1357 - loss 6.984357833862305 - moving ave loss 9.124147247403691
step 1

### Prediction
If you want to predict all pictures in the _sample_images_ directory for testing use this shell command (in the darkflow directory):
```   
python flow --imgdir ..\TrafficLightDetector\sample_images --model ..\TrafficLightDetector\cfg\yolo_bosch.cfg --load -1
```

In [16]:
from darkflow.net.build import TFNet
import cv2
import time

options = {"model": "cfg/yolo_bosch.cfg",
           "load": -1,
           "threshold": 0.1}

tfnet_cpu = TFNet(options)
options["gpu"] = 1.0
tfnet_gpu = TFNet(options)

imgcv = cv2.imread("./sample_images/489562.png")
print("all loaded")

Parsing cfg/yolo_bosch.cfg
Loading None ...
Finished in 0.0005009174346923828s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 16)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 26, 26, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 26, 26, 256)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 13, 13, 2

In [17]:
# predict with cpu options
cpu_t1 = time.time()
result = tfnet_cpu.return_predict(imgcv)
cpu_t2 = time.time()

# predict with gpu options
gpu_t1 = time.time()
result = tfnet_gpu.return_predict(imgcv)
gpu_t2 = time.time()

In [18]:
ct = cpu_t2-cpu_t1
gt = gpu_t2-gpu_t1
print("CPU prediction time: {:.5f} s, Rate: {:.2f} Hz".format(ct, 1/ct))
print("GPU prediction time: {:.5f} s, Rate: {:.2f} Hz".format(gt, 1/gt))

CPU prediction time: 0.25966 s, Rate: 3.85 Hz
GPU prediction time: 0.05414 s, Rate: 18.47 Hz
